In [ ]:
from dataclasses import dataclass
import opt_note.scsp as scsp

In [ ]:
import marimo as mo
import nbformat

# LA-SH モデルと WMM を組み合わせる

LA-SH では $m$ 手進んだときに最も良い選択をするが,
より先を見て判断するため, 進んだ手数の代わりに Weight の和を採用する.

$m = 1$, $l = 1$ のときは WMM と同じである.

In [ ]:
def find_next_strategy(
    instance: list[str],
    chars: str,
    state: tuple[int, ...],
    m: int,
) -> tuple[str, int]:
    """
    現在の状態を受け取り, m 手進めたときに weight の和が最大になる文字の選び方 (長さ m の文字列として表される) と sum-weight の値を組みにして返す
    """

    if m == 0 or all(idx == len(s) for idx, s in zip(state, instance)):
        return ("", 0)

    fronts = [s[idx] for idx, s in zip(state, instance) if idx < len(s)]
    counts = {char: fronts.count(char) for char in chars}
    weights = {
        char: sum(
            len(s) - idx for idx, s in zip(state, instance) if idx < len(s) and s[idx] == char
        ) for char in chars
    }
    max_sum_weight = 0
    max_str_front = ""
    explores = set()
    for char in chars:
        if char not in fronts or char in explores:
            continue
        explores.add(char)
        ahead_state = tuple(
            idx + 1 if idx < len(s) and s[idx] == char else idx
            for idx, s in zip(state, instance)
        )
        str_ahead, sum_ahead = find_next_strategy(instance, chars, ahead_state, m - 1)
        # if sum_ahead + counts[char] > max_sum_weight:
        #     max_sum_weight = sum_ahead + counts[char]
        #     max_str_front = char + str_ahead
        if sum_ahead + weights[char] > max_sum_weight:
            max_sum_weight = sum_ahead + weights[char]
            max_str_front = char + str_ahead
    return (max_str_front, max_sum_weight)

In [ ]:
@dataclass
class Model:
    instance: list[str]
    solution: str | None = None
    best_bound: float = 0.0

    def solve(self, m: int = 3, ll: int = 1, *args, **kwargs) -> str:
        chars = "".join(sorted(list(set("".join(self.instance)))))
        state = tuple(0 for _ in self.instance)
        solution = ""

        while not all(idx == len(s) for idx, s in zip(state, self.instance)):
            next_str, _ = find_next_strategy(self.instance, chars, state, m)
            solution += next_str[:ll]
            for next_char in next_str[:ll]:
                state = tuple(
                    idx + 1 if idx < len(s) and s[idx] == next_char else idx
                    for idx, s in zip(state, self.instance)
                )

        self.solution = solution
        return self.solution

In [ ]:
scsp.util.bench(Model, example_filename="uniform_q26n004k015-025.txt")

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 68) ---
 Sol: itkgonjikqefoulcinbvahmpxxyczgobrddnbcsohvstuqpgoxrzvxinpnpsglsbpfxf
str1: -tkg-n--k----u-------hmpx----------n----h--t-q-g-x-zvxi----s--------
str2: i---o-ji-q-fo-l--nb-----xx-c-------------vs-uqp-----v-i----s--sb--xf
str3: -------------ulcin--------yc--o-------so-v------o--z----p-p--l--p---
str4: i--g------e--------va-------zg-brdd-bcs--v--------r-v--n-n--g----f--

example file name: 'uniform_q26n004k015-025.txt'
best objective: 68
best bound: 0.0
wall time: 0.028912s


In [ ]:
scsp.util.bench(scsp.model.la_sh.Model, example_filename="uniform_q26n004k015-025.txt")

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 70) ---
 Sol: itkgnekojiqfouhlcimpnbvaxxnhtqcgxycozgbrddbcsovxiorsuqzpplpvinngfssbxf
str1: -tkgn-k------uh---mp----x-nhtq-gx---z---------vxi--s------------------
str2: i------ojiqfo--l----nb--xx----c---------------v----suq-p---vi----ssbxf
str3: -------------u-lci--n------------yco--------sov--o----zpplp-----------
str4: i--g-e----------------va------------zgbrddbcs-v---r--------v-nngf-----

example file name: 'uniform_q26n004k015-025.txt'
best objective: 70
best bound: 0.0
wall time: 0.007959s


In [ ]:
scsp.util.bench(Model, example_filename="uniform_q26n008k015-025.txt")

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 111) ---
 Sol: igenopbdevdaczfjtrvkgxdiypwxlqnfkrzoxulcinbdehprmpqlxxycodvgstnhodbftmuqipcsbvrogxqzbrpvxcdisgplnnbgsbwdehopfxf
str1: ----------------t--kg---------n-k----u-------h--mp--x---------nh----t--q--------gx-z---vx--is------------------
str2: i---o----------j-------i-----q-f---o--l--nb---------xx-c--v-s---------uq-p---v-------------is-------sb-------xf
str3: -------------------------------------ulcin------------yco---s---o------------v-o---z--p-------pl-----------p---
str4: ige------v-a-z------g---------------------b----r---------d-------db-------cs-vr--------v--------nn-g--------f--
str5: -----p------------------yp--l----rz-xu-c------p-m-q-------vg-t---d-f--u-i----v-----------cd-s-----b------

In [ ]:
scsp.util.bench(scsp.model.la_sh.Model, example_filename="uniform_q26n008k015-025.txt")

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 132) ---
 Sol: eingeojipbdcevaqdcrtkzfgjtbnkuhlcimpvxdnerddbycopflrhnzbsmorvigoplersbrowtxqgxckrdrlctodtmzprplpucpmqvxbbghinngfstdfuiqpvcdissbowdxf
str1: -------------------tk--g---nkuh---mp-x-n------------h--------------------t-qgx------------z----------vx----i----s-------------------
str2: -i---oji-------q------f------------------------o--l--n-b------------------x--xc----------------------v----------s---u-qpv--issb---xf
str3: -----------------------------u-lci-----n-----yco--------s-o-v--o--------------------------zp-plp------------------------------------
str4: -i-ge--------va------z-g--b--------------rddb-c---------s---v------r---------------------------------v------nngf--------------------
str5: 

In [ ]:
scsp.util.bench(Model, example_filename="uniform_q26n016k015-025.txt")

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 153) ---
  Sol: kprisxtkgenbqawxudsflcojignqfykzpehvalrdzxuhecoxpmqsabivglnbrtdpfxjulxctovdwozsiqbkxaenhtjmruzqpswbrfgvcihkdjosxqpmgzkvrstbvlnpahngqxbfiklmwwzceghnosuvdy
str01: ------tkg-n-------------------k-----------uh-----m-------------p-x--------------------nht-----q------g---------x----z-v-------------x--i------------s----
str02: ---i------------------oji--qf-----------------o----------lnb-----x---xc--v----s-------------u-qp------v-i-----s---------s-b---------x-f-----------------

In [ ]:
scsp.util.bench(scsp.model.la_sh.Model, example_filename="uniform_q26n016k015-025.txt")

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 170) ---
  Sol: kpriflshtkqagenbkvxuwdgsfchxeqjafnvzfmpacdigqbcejtivwbahcokrdxerlcnvpfzksgjmiudbcrsvihtjodqbgkderwxzavntfmoybplngqrbpzvxucbbhefikoshmgopmqvgtowdfuivxcdvsbouqyzpplpvissbxf
str01: --------tk--g-n-k--u------h----------mp----------------------x----n------------------ht---q-g-----xz-v-----------------x-------i--s---------------------------------------
str02: ---i-----------------------------------------------------o----------------j-i-------------q-------------f-o---ln---b--

In [ ]:
scsp.util.bench(Model, example_filename="uniform_q05n010k010-010.txt")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 31) ---
  Sol: abdcbacdbeecdaebdecbaddacebddee
str01: --dcb-c----cd--b--c-----ce-----
str02: -bd----dbee---e--e-b-d---------
str03: ---c-acd-eec--eb-e-------------
str04: a--------e--d---d----dd--ebdd--
str05: a--cb----eec-a-b--c------e-----
str06: -b--ba--be-----bd-cba----------
str07: -b--ba---e---aeb----ad-a-------
str08: ---------ee---e--ecb-d----b--ee
str09: ---c--cd-ee-da--d-c--d---------
str10: -bd--a--b---d--b-e--a--a---d---

example file name: 'uniform_q05n010k010-010.txt'
best objective: 31
best bound: 0.0
wall time: 0.007537s


In [ ]:
scsp.util.bench(scsp.model.la_sh.Model, example_filename="uniform_q05n010k010-010.txt")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 31) ---
  Sol: abdcbacdbeecdaebdecbadabcdebdde
str01: --dcb-c----cd--b--c-----c-e----
str02: -bd----dbee---e--e-b-d---------
str03: ---c-acd-eec--eb-e-------------
str04: a--------e--d---d----d---debdd-
str05: a--cb----eec-a-b--c-------e----
str06: -b--ba--be-----bd-cba----------
str07: -b--ba---e---aeb----ada--------
str08: ---------ee---e--ecb-d-b--e---e
str09: ---c--cd-ee-da--d-c--d---------
str10: -bd--a--b---d--b-e--a-a--d-----

example file name: 'uniform_q05n010k010-010.txt'
best objective: 31
best bound: 0.0
wall time: 0.004314s


In [ ]:
scsp.util.bench(Model, example_filename="uniform_q05n050k010-010.txt")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 36) ---
  Sol: dacebdacebdabceaedcbadeabcd

In [ ]:
scsp.util.bench(scsp.model.la_sh.Model, example_filename="uniform_q05n050k010-010.txt")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 38) ---
  Sol: dacbedabcedabcdeaecbdacebad

In [ ]:
scsp.util.bench(Model, example_filename="nucleotide_n010k010.txt")

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 26) ---
  Sol: TCATACCGGGATATCGATCGTAAACT
str01: --AT---GGGATA-CG----------
str02: --ATACC----T-TC---C-----C-
str03: -CA--C-G--A-AT---T-G-A----
str04: T-A-A-----A-ATC--T-GT-----
str05: --A----GG--TA---A-C--AAA--
str06: T--T-CC----TA--G---GTA----
str07: T--T---G---TA--GATC-T-----
str08: T------GGGA-A--G-T--T---C-
str09: T--T-CC---A---C-A----A--CT
str10: TC-TA-----A-A-CGA----A----

example file name: 'nucleotide_n010k010.txt'
best objective: 26
best bound: 0.0
wall time: 0.003961s


In [ ]:
scsp.util.bench(scsp.model.la_sh.Model, example_filename="nucleotide_n010k010.txt")

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 28) ---
  Sol: TATACACGGGATACGAATCGATCACAGA
str01: -AT----GGGATACG-------------
str02: -ATAC-C----T-----TC---C-C---
str03: ----CACG--A-A----T---T----GA
str04: TA-A-A----AT-C---T-G-T------
str05: -A-----GG--TA--A--C-A--A-A--
str06: T-T-C-C----TA-G----G-T-A----
str07: T-T----G---TA-GA-TC--T------
str08: T------GGGA-A-G--T---TC-----
str09: T-T-C-C---A--C-AA-C--T------
str10: T---C------TA--AA-CGA--A----

example file name: 'nucleotide_n010k010.txt'
best objective: 28
best bound: 0.0
wall time: 0.002543s


In [ ]:
scsp.util.bench(Model, example_filename="nucleotide_n050k050.txt")

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

In [ ]:
scsp.util.bench(scsp.model.la_sh.Model, example_filename="nucleotide_n050k050.txt")

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

In [ ]:
scsp.util.bench(Model, example_filename="protein_n010k010.txt")

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 51) ---
  Sol: MEQSKAFPLSVRNYAEQHAFGLVNDTAICPGERKFSLRNGAHPDELTVQYQ
str01: M----A--LS---Y--------------CP---K-----G------T----
str02: M-QS-----S-----------L-N--AI-P-----------------V---
str03: M------PLS---Y--QH-F------------RK-----------------
str04: ME-------------E-H----VN-------E----L----H-D-------
str05: M--S--------N------F----D-AI----R-------A----L-----
str06: M-----F----RN---Q------N-----------S-RNG-----------
str07: M-----F------YA--HAFG---------G------------------Y-
str08: M--SK-F------------------T------R----R----P------YQ
str09: M--S--F---V---A-----G-V--TA---------------------Q--
str10: ME-S----L-V------------------PG---F---N-----E------

example file name: 'protein_n010k010.txt'
best objective: 51
best bound: 0.0
wall time: 0.104955s


In [ ]:
scsp.util.bench(scsp.model.la_sh.Model, example_filename="protein_n010k010.txt")

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 51) ---
  Sol: MEQSKAFPLSVEYACHPGFLRVNTAEQHFRDAIKGGRALHDNPSRNGTVYQ
str01: M----A--LS--Y-C-P----------------KG------------T---
str02: M-QS-----S---------L--N-A-------I---------P-----V--
str03: M------PLS--Y-------------QHFR---K-----------------
str04: ME---------E---H-----VN--E------------LHD----------
str05: M--S------------------N-----F-DAI---RAL------------
str06: M-----F-------------R-N---Q--------------N-SRNG----
str07: M-----F-----YA-H--------A---F-----GG-------------Y-
str08: M--SK-F----------------T-----R------R-----P------YQ
str09: M--S--F---V--A---G---V-TA-Q------------------------
str10: ME-S----L-V-----PGF---N--E-------------------------

example file name: 'protein_n010k010.txt'
best objective: 51
best bound: 0.0
wall time: 0.035093s


In [ ]:
scsp.util.bench(Model, example_filename="protein_n050k050.txt")

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS

In [ ]:
scsp.util.bench(scsp.model.la_sh.Model, example_filename="protein_n050k050.txt")

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS